In [2]:
from math import sqrt
import numpy as np
import cirq
import openfermion as of

In [10]:
def eps2_bound(hamiltonian: cirq.PauliSum) -> float:
    """Get a bound on eps2."""

    M = len(hamiltonian)
    coeffs = np.array([ps.coefficient for ps in hamiltonian])
    i_max = np.argmax(np.abs(coeffs))
    max_coeff = coeffs[i_max]
    n_summands = M
    return (-1. / 24) * 0.5 * max_coeff ** 3 * n_summands ** 3

## Bounds for Hubbard

In [11]:
energy_error = 1e-3

In [13]:
t = 1.0
u = 4.0
ls = [2, 3, 4]
eps2_bounds = []
dt_bounds = []
for l in ls:
    ham = of.fermi_hubbard(l, l, t, u, spinless=True)
    ham_jw = of.transforms.jordan_wigner(ham)
    ham_cirq = of.transforms.qubit_operator_to_pauli_sum(ham_jw)
    bound = eps2_bound(ham_cirq)
    eps2_bounds.append(bound)
    dt_bounds.append(sqrt(energy_error / abs(bound)))
for l, dt in zip(ls, dt_bounds):
    print(f"{l} {dt:4.5e}")

2 3.90712e-04
3 5.60327e-06
4 1.00758e-06
